# 主双対法による制約付きMDPの解法

参考

* [Exploration-Exploitation in Constrained MDPs](https://arxiv.org/abs/2003.02189)
* [CONSTRAINED MARKOV DECISION PROCESSES](https://www-sop.inria.fr/members/Eitan.Altman/TEMP/h.pdf)

[前回](RL_CMDP_explore_exploit.ipynb)は線型計画法によってCMDPを解く方法を見ました．
今回は主双対法をみます．

![OptDual](figs/OptPrimalDual-CMDP.png)


準備：
* Optimisticな制約違反： $\widetilde{g}_k =\left({D}_k q^{\pi_k}\left(\widetilde{p}_k\right)-\alpha\right)$
* 真の制約違反： $g_k =\left(D q^{\pi_k}(p)-\alpha\right)$
* Optimisticな価値：$\tilde{f}_k={c}_k^T q^{\pi_k}\left(\widetilde{p}_k\right)$
* 真の価値：$f_k=c^T q^{\pi_k}$
* 最適価値：$f_{o p t}=V_1^*\left(s_1\right)=c^T q^*$

また，次のFailure Eventを考えます（LPのときとだいたい同じです）．

$$
\begin{aligned}
& F_k^p=\left\{\exists s, a, s^{\prime}, h:\left|p_h\left(s^{\prime} \mid s, a\right)-\bar{p}_h^{k-1}\left(s^{\prime} \mid s, a\right)\right| \geq \beta_{h, k}^p\left(s, a, s^{\prime}\right)\right\} \\
& F_k^N=\left\{\exists s, a, h: n_h^{k-1}(s, a) \leq \frac{1}{2} \sum_{j<k} q_h^{\pi_k}(s, a \mid p)-H \ln \frac{S A H}{\delta^{\prime}}\right\}
\end{aligned}
$$

前回同様，Failure Eventに対して、
$\bar{G}=F^p \bigcup F^N$の補集合としてGood Event$G$を定義します．
このGood Eventが成り立っているとき、[RL_UCB_VI_regret_proof.ipynb](RL_UCB_VI_regret_proof.ipynb)でやったような分散の変形を使うと，

$$
\left|\bar{p}_h^{k-1}\left(s^{\prime} \mid s, a\right)-p_h\left(s^{\prime} \mid s, a\right)\right|=C_1 \sqrt{\frac{p_h\left(s^{\prime} \mid s, a\right) L_{\delta, p}}{n_h^k(s, a) \vee 1}}+\frac{C_2 L_{\delta, p}}{n_h^k(s, a) \vee 1}
$$

となる定数$C_1, C_2 > 0$が存在します。ここで$L_{\delta, p}=\ln \left(\frac{6 S A H K}{\delta}\right)$です。

これをつかってボーナスを報酬とコスト関数に乗せて，まず方策の評価をします：

* $\left\{Q_h^{\pi_k}\left(s, a ; \widetilde{c}_k, \bar{p}_{k-1}\right)\right\}_{s, a, h}=$ Trun. Policy Evaluation $\left(\widetilde{c}_k, \bar{p}_{k-1}, \pi_k\right)$
* $\forall i \in[I],\left\{Q_h^{\pi_k}\left(s, a ; \widetilde{d}_{i, k}, \bar{p}_{k-1}\right)\right\}_{s a h}=$ Trun. Policy Evaluation $\left(\widetilde{d}_i^k, \bar{p}_{k-1}, \pi_k\right)$

続いて，方策の更新を鏡像降下で実現します：

$$
\begin{aligned}
& Q_h^k(s, a)=Q_h^{\pi_k}\left(s, a ; \widetilde{c}_k, \bar{p}_{k-1}\right)+\sum_{i=1}^I \lambda_{k, i} Q_h^{\pi_k}\left(s, a ; \widetilde{d}_{k, i}, \bar{p}_{k-1}\right) \\
& \pi_h^{k+1}(a \mid s)=\frac{\pi_h^k(a \mid s) \exp \left(-t_K Q_h^k(s, a)\right)}{\sum_{a^{\prime}} \pi_h^k\left(a^{\prime} \mid s\right) \exp \left(-t_K Q_h^k\left(s, a^{\prime}\right)\right)}
\end{aligned}
$$



## ステップ１：On policy Optimality

Good eventが成立しているとき，任意の$k \in [K]$について，
$$
\sum_{k=1}^{K^{\prime}} \widetilde{f}_k+\lambda_k^T \widetilde{g}_k-f_{\pi^*}-\lambda_k^T g_{\pi^*} \leq \widetilde{\mathcal{O}}\left(\sqrt{H^4(1+I \rho)^2 K}\right)
$$

が成り立ちます．

**証明**

定義より，

$$
\begin{aligned}
& f_{\pi^*}+\lambda_k^T g_{\pi^*}=V_1^{\pi^*}\left(s_1 ; c, p\right)+\sum_{i=1}^I \lambda_{k, i} \left(V_1^{\pi^*}\left(s_1 ; d_i, p\right)- \alpha_i \right)\\
& \widetilde{f}_k+\lambda_k^T \widetilde{g}_k=\widehat{V}_1^{\pi_k}\left(s_1 ; \widetilde{c}_k, \bar{p}_k\right)+\sum_{i=1}^I \lambda_{k, i} \left(\widehat{V}_1^{\pi_k}\left(s_1 ; \widetilde{d}_{k, i}, \bar{p}_k\right)-\alpha_i\right) .
\end{aligned}
$$

です．

また，

$$
\begin{aligned}
Q_h^k(s, a) & :=Q_h^{\pi_k}\left(s, a ; \widetilde{c}_k, \bar{p}_{k-1}\right)+\sum_{i=1}^I \lambda_{k, i} Q_h^{\pi_k}\left(s, a ; \widetilde{d}_{k, i}, \bar{p}_{k-1}\right) \\
V_h^k\left(s_1\right) & :=\left\langle Q_h^k(s, \cdot), \pi_h^k\right\rangle .
\end{aligned}
$$

とします．ここで，$V-V$を変形して，

$$
\begin{aligned}
& \sum_{k=1}^{K^{\prime}} \widetilde{f}_k+\lambda_k^T \widetilde{g}_k-f_{\pi^*}-\lambda_k^T g_{\pi^*} \\
& =\sum_{k=1}^{K^{\prime}} V_1^k\left(s_1\right)-V_1^{\pi^*}\left(s_1 ; c+\lambda_k \widetilde{d}, p\right) \\
& =\underbrace{\sum_{k=1}^K \sum_{h=1}^H \mathbb{E}\left[\left\langle Q_h^k\left(s_h, \cdot\right), \pi_h^k\left(\cdot \mid s_h\right)-\pi_h^*\left(\cdot \mid s_h\right)\right\rangle \mid s_1=s_1, \pi^*, p\right]}_{(i)} \\
& +\sum_{k=1}^K \sum_{h=1}^H \mathbb{E}[\underbrace{Q_h^k\left(s_h, a_h\right)-c_h\left(s_h, a_h\right)-\sum_{i=1}^I \lambda_k d_{h, i}\left(s_h, a_h\right)-p_h\left(\cdot \mid s_h, a_h\right) V_{h+1}^k}_{(i i)} \mid s_1=s_1, \pi^*, p] .
\end{aligned}
$$

を得ます．(i)と(ii)をそれぞれバウンドします．

**(i)のバウンド（論文のLemma 25）**

これは[RL_useful_lemma.ipynb](RL_useful_lemma.ipynb)の **補題：オンラインのMirror descentをするときに便利**を使うと，

$$
\sum_{k=1}^K\left\langle Q_h^k(\cdot \mid s), \pi_h^k(\cdot \mid s)-\pi_h(\cdot \mid s)\right\rangle \leq \frac{\log A}{t_K}+\frac{t_K}{2} \sum_{k=1}^K \sum_a \pi_h^k(a \mid s)\left(Q_h^k(s, a)\right)^2
$$

であり，ここに$Q_h^k(s, a):=Q_h^{\pi_k}\left(s, a ; \widetilde{c}_k, \bar{p}_{k-1}\right)+\sum_{i=1}^I \lambda_{k, i} Q_h^{\pi_k}\left(s, a ; \widetilde{d}_{k, i}, \bar{p}_{k-1}\right)$を代入すれば，

$$
(i)=\sum_{k=1}^{K^{\prime}} \sum_{h=1}^H \mathbb{E}\left[\left\langle Q_h^k\left(s_h, \cdot\right), \pi_h^k\left(\cdot \mid s_h\right)-\pi_h^*\left(\cdot \mid s_h\right)\right\rangle \mid s_1=s_1, \pi^*, p\right] \lesssim \sqrt{H^4(1+I \rho)^2 K}
$$

であることがすぐにわかります．

**(ii)のバウンド（論文のLemma 26）**

こっちはOptimismを使うと０以下に抑えられます．（TODO: 詳細）

## ステップ２：Dual Optimism

こっから先は[RL_CMDP_explore_exploit_dual.ipynb](RL_CMDP_explore_exploit_dual.ipynb)と似たことをします．まず，

$$
\widetilde{f}_k-f_{o p t} \leq-\lambda_k^T \widetilde{g}_k+\left(\widetilde{f}_k+\lambda_k^T \widetilde{g}_k-f_{\pi^*}-\lambda_k^T g_{\pi^*}\right)
$$

を示します．これもだいたい同じなので省略．
後はだいたい同じです．